In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import plotnine

from plotnine import *

# get param set
paramsets =  pd.read_csv('../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
outputvalsname =  ['wat','cs','cl','coa','cod','cca',
                   'ccd','css2','co2','yX','yY','yZ']
column_names = outputvalsname + paramsets.columns.tolist()
paramsets.drop('theta',inplace=True, axis=1)

df = pd.read_pickle("./cyl10to11_1d1dF.pkl")  # cid, pSet, rr, 1 or 9


numPset = 99
nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16
coordId = 17
nToGet = coordId+1 #elements + 3d coordinates of y , node+vol+len+st

In [3]:
df.shape

(16, 99)

In [2]:
konzcs = df.loc[0]# pSet, rr, 9
vols = df.loc[volIdx]# pSet, rr, 9
lens = df.loc[lenIdx]# pSet, rr, 1
orgIds = df.loc[orgIdId]# pSet, rr, 1
reLens =  df.loc[relLenId]
coords =  df.loc[coordId]

KeyError: 16

In [ ]:
coords[pSet][1]

In [84]:
orgIds

0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                            ...                        
94    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
95    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
96    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
97    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
98    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: 15, Length: 99, dtype: object

In [52]:
repr([sum(orgIds.loc[pSet]==1) for pSet in range(99) ])

'[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]'

In [55]:
dfnew = pd.DataFrame(columns=['locCylId','coords','konzcs','vols','lens','reLens','pSet'])

for pSet in range(99):
    
    cyltr = sum(orgIds.loc[pSet]==1)# 21 or 22
    locCylId = np.array([np.full(9,ii) for ii in range(cyltr)])
    locCellId = np.full((cyltr,9),np.array([i for i in range(9)]))
    
    myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})
    
    myData['locCellId']= locCellId.reshape(-1)
    myData['coords']= coords.loc[pSet][orgIds.loc[pSet]==1].reshape(-1)
    myData['konzcs']= konzcs.loc[pSet][orgIds.loc[pSet]==1].reshape(-1)
    myData['vols']= vols.loc[pSet][orgIds.loc[pSet]==1].reshape(-1)
    
    lens__ = lens.loc[pSet][orgIds.loc[pSet]==1]
    lens_ = np.array([np.full(9, ll) for ll in lens__])
    reLens__ = reLens.loc[pSet][orgIds.loc[pSet]==1]
    reLens_ = np.array([np.full(9, ll) for ll in reLens__])
    myData['lens']= lens_.reshape(-1)
    myData['reLens']= reLens_.reshape(-1)
    myData['pSet'] = pSet
    
    dfnew = pd.concat([dfnew, myData], ignore_index=True)
    

In [71]:
dfnew['rIn'] = -1
dfnew['rOut'] = -1
dfnew.loc[dfnew['locCellId']==0,'rIn'] = 0.05
for pSet in range(99):
    cyltr = sum(orgIds.loc[pSet]==1)
    for cylid in range(cyltr):
        #lenCyl = dfnew[dfnew['pSet']==pSet][dfnew['locCylId']==cylid]['rIn'] 
        for cellid in range(1,9):
            rowcell = dfnew.loc[(dfnew['pSet']==pSet)&(dfnew['locCylId']==cylid)&(dfnew['locCellId']==cellid)].index.tolist()[0]
            lencyl = dfnew.at[rowcell,'lens']
            vol =  dfnew.at[rowcell,'vols']
            dfnew.at[rowcell,'rOut'] = np.sqrt(vol/(np.pi*lencyl) +dfnew.at[rowcell,'rIn']**2 )
            
            if cellid < 8:
                rownextcell = dfnew.loc[(dfnew['pSet']==pSet)&(dfnew['locCylId']==cylid)&(dfnew['locCellId']==0)].index.tolist()[0]
                dfnew.at[rownextcell,'rIn'] = dfnew.at[rowcell,'rOut']


In [79]:
dfnew[dfnew['coords'] < 0.05]

,locCylId,coords,konzcs,vols,lens,reLens,pSet,locCellId,rIn,rOut
9,1,0.022022,0.000147,0.000560,1.0,2.0,0,0.0,1.001173,-1.000000
10,1,0.026476,0.000212,0.000809,1.0,2.0,0,1.0,-1.000000,1.000129
11,1,0.031830,0.000306,0.001169,1.0,2.0,0,2.0,-1.000000,1.000186
12,1,0.038266,0.000442,0.001690,1.0,2.0,0,3.0,-1.000000,1.000269
13,1,0.046005,0.000640,0.002442,1.0,2.0,0,4.0,-1.000000,1.000389
...,...,...,...,...,...,...,...,...,...,...
18704,19,0.037477,0.000469,0.002178,1.0,20.0,98,2.0,-1.000000,1.000347
18705,19,0.048031,0.000770,0.003578,1.0,20.0,98,3.0,-1.000000,1.000569
18711,20,0.023422,0.000224,0.001007,1.0,21.0,98,0.0,1.009828,-1.000000
18712,20,0.031438,0.000404,0.001815,1.0,21.0,98,1.0,-1.000000,1.000289


In [72]:
dfnew.loc[dfnew['locCellId']==0,'coords']# = 0.05

0        0.054990
9        0.022022
18       0.022174
27       0.022305
36       0.022305
           ...   
18675    0.023947
18684    0.023947
18693    0.057473
18702    0.022816
18711    0.023422
Name: coords, Length: 2080, dtype: float64

In [74]:
dfnew.to_pickle('./cyl10to11_celss2.pkl')

In [77]:
print(dfnew.loc[dfnew['locCellId']==0,'coords'].min(),dfnew.loc[dfnew['locCellId']==0,'coords'].max())
print(dfnew.loc[dfnew['locCellId']==8,'coords'].min(),dfnew.loc[dfnew['locCellId']==8,'coords'].max())

0.02202224498672597 0.057472616323468714
0.09610504338489201 0.47702422259471233


In [ ]:
dist = 0.0

In [ ]:
dfnew.loc[dfnew['locCellId']==0,'rIn']

In [ ]:
plt.scatter(dfnew['yX'],df['yZ'] , #np.log(df['cc']),np.log(df['co']), 
            c= np.log(df['cs']),#s=df['cs'],
            alpha=0.3, edgecolors ='none')
plt.rcParams.update({'figure.figsize':(5,5), 'figure.dpi':100})
#plt.title()
plt.colorbar()
plt.xlabel('cs')
plt.ylabel('co +cc')
plt.show()

In [41]:

cyltr = sum(orgIds.loc[pSet]==1)# 21
locCylId = np.array([np.full(9,ii) for ii in range(cyltr)])
locCellId = np.full((21,cyltr),np.array([i for i in range(cyltr)]))

myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})

#myData['locCellId']= locCellId.reshape(-1)

In [49]:
locCylId.shape

(22, 9)

In [50]:
locCellId.shape

(21, 9)

In [33]:
lens__ = lens.loc[pSet][orgIds.loc[0]==1]

In [36]:
lens_ = np.array([np.full(9, ll) for ll in lens__])

In [37]:
lens_

array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [21]:
cc.shape

(21, 9)

In [26]:
cc[0]

array([0.05499005, 0.06596616, 0.07913313, 0.09492826, 0.11387611,
       0.136606  , 0.16387281, 0.19658214, 0.2358203 ])

In [25]:
cc.reshape(-1)

array([0.05499005, 0.06596616, 0.07913313, 0.09492826, 0.11387611,
       0.136606  , 0.16387281, 0.19658214, 0.2358203 , 0.02202224,
       0.02647568, 0.03182971, 0.03826646, 0.04600488, 0.05530819,
       0.06649286, 0.07993935, 0.09610504, 0.02217415, 0.02699516,
       0.03286432, 0.04000953, 0.04870822, 0.05929814, 0.07219047,
       0.08788579, 0.10699352, 0.02230469, 0.02744521, 0.03377047,
       0.04155351, 0.05113028, 0.0629142 , 0.07741395, 0.09525543,
       0.11720881, 0.02230469, 0.02744521, 0.03377047, 0.04155351,
       0.05113028, 0.0629142 , 0.07741395, 0.09525543, 0.11720881,
       0.02202224, 0.02647568, 0.03182971, 0.03826646, 0.04600488,
       0.05530819, 0.06649286, 0.07993935, 0.09610504, 0.0227442 ,
       0.02898565, 0.03693989, 0.04707692, 0.05999575, 0.07645977,
       0.09744183, 0.12418179, 0.15825972, 0.02363763, 0.03223614,
       0.04396248, 0.05995442, 0.08176365, 0.11150628, 0.15206819,
       0.20738504, 0.28282415, 0.0227442 , 0.02898565, 0.03693